이제 모델링을 해볼 차례.\
여기서 성능이 원하는 만큼 나오지 않으면 다시 EDA를 해야함.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
import xgboost
from xgboost import XGBClassifier

In [2]:
plt.rcParams['font.family'] = 'NanumGothic'

In [3]:
df = pd.read_csv('./train_delete_outlier.csv')
df

,연간 소득,신용 거래 연수,월 상환 부채액,최대 신용한도,현재 미상환 신용액,개설된 신용계좌 수,현재 대출 잔액,마지막 연체 이후 경과 개월 수,신용 점수,체납 세금 압류 횟수,신용 문제 발생 횟수,개인 파산 횟수,credit_rating_range,current_range_employment,purpose,house_type,채무 불이행 여부,loan repayment period
0,1941337.5,13.4,8806.5,400597.5,225457.5,9.0,390903.0,24.0,767.0,0.0,0,1,700,10,14,0,0,0
1,1356381.0,18.8,12069.0,491770.5,487644.0,12.0,227775.0,74.0,800.0,0.0,1,3,700,4,14,1,1,0
2,1049017.5,14.8,31749.0,411546.0,413211.0,15.0,251383.5,22.0,796.0,0.0,1,1,700,6,14,1,1,0
3,1410700.5,24.4,16842.0,692352.0,154974.0,8.0,479349.0,19.0,807.0,0.0,0,0,800,2,13,2,0,1
4,1636863.0,9.8,28818.0,154429.5,11943.0,9.0,370210.5,11.0,713.0,0.0,0,0,700,3,12,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,1726114.5,18.3,5043.0,1035360.0,16635.0,8.0,280647.0,24.0,731.0,1.0,5,0,700,1,14,0,0,0
5984,1294200.0,17.5,6700.5,266356.5,12522.0,13.0,262972.5,7.0,798.0,5.0,0,0,700,2,10,0,0,0
5985,1327737.0,14.2,8163.0,100882.5,141427.5,10.0,480877.5,32.0,649.0,0.0,0,0,600,7,14,0,0,1
5986,1198722.0,17.4,3919.5,683016.0,383376.0,10.0,521730.0,25.0,742.0,0.0,1,1,700,8,14,1,1,0


In [4]:
x = df.drop(labels=['채무 불이행 여부'], axis = 1)
x

,연간 소득,신용 거래 연수,월 상환 부채액,최대 신용한도,현재 미상환 신용액,개설된 신용계좌 수,현재 대출 잔액,마지막 연체 이후 경과 개월 수,신용 점수,체납 세금 압류 횟수,신용 문제 발생 횟수,개인 파산 횟수,credit_rating_range,current_range_employment,purpose,house_type,loan repayment period
0,1941337.5,13.4,8806.5,400597.5,225457.5,9.0,390903.0,24.0,767.0,0.0,0,1,700,10,14,0,0
1,1356381.0,18.8,12069.0,491770.5,487644.0,12.0,227775.0,74.0,800.0,0.0,1,3,700,4,14,1,0
2,1049017.5,14.8,31749.0,411546.0,413211.0,15.0,251383.5,22.0,796.0,0.0,1,1,700,6,14,1,0
3,1410700.5,24.4,16842.0,692352.0,154974.0,8.0,479349.0,19.0,807.0,0.0,0,0,800,2,13,2,1
4,1636863.0,9.8,28818.0,154429.5,11943.0,9.0,370210.5,11.0,713.0,0.0,0,0,700,3,12,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,1726114.5,18.3,5043.0,1035360.0,16635.0,8.0,280647.0,24.0,731.0,1.0,5,0,700,1,14,0,0
5984,1294200.0,17.5,6700.5,266356.5,12522.0,13.0,262972.5,7.0,798.0,5.0,0,0,700,2,10,0,0
5985,1327737.0,14.2,8163.0,100882.5,141427.5,10.0,480877.5,32.0,649.0,0.0,0,0,600,7,14,0,1
5986,1198722.0,17.4,3919.5,683016.0,383376.0,10.0,521730.0,25.0,742.0,0.0,1,1,700,8,14,1,0


In [5]:
y = df[['채무 불이행 여부']]
y

,채무 불이행 여부
0,0
1,1
2,1
3,0
4,1
...,...
5983,0
5984,0
5985,0
5986,1


In [6]:
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.7, random_state = 42) 

In [7]:
train_y

,채무 불이행 여부
1187,0
5081,1
2646,1
1030,1
2244,0
...,...
3772,1
5191,1
5226,1
5390,0


In [8]:
dtrain = xgboost.DMatrix(data = train_x, label = train_y)
dtest = xgboost.DMatrix(data = test_x, label = test_y)

In [9]:
xgbc = XGBClassifier()
xgbc_params = {
    'n_estimators':[100, 200, 300, 400, 500],
    'learning_rate':[0.01, 0.05, 0.1],
    'max_depth':[2, 3, 4, 5]
}

In [ ]:
xgb_grid = GridSearchCV(xgbc, param_grid=xgbc_params, scoring = 'roc_auc', cv = 5)
xgb_grid.fit(train_x, train_y)